In [2]:
!pip install tensorflow
!pip install contractions
import tensorflow as tf
import pandas as pd
import numpy as np

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 9.2 MB/s eta 0:00:00


In [10]:
train_df = pd.read_csv("train.csv")
train_df.head()

,Score,Text
0,5,I received this product early from the seller!...
1,5,*****<br />Numi's Collection Assortment Melang...
2,5,"I was very careful not to overcook this pasta,..."
3,5,Buying this multi-pack I was misled by the pic...
4,5,These bars are so good! I loved them warmed up...


In [11]:
import string
train_df['Text_Processed'] = train_df['Text'].str.lower()
train_df.head()

,Score,Text,Text_Processed
0,5,I received this product early from the seller!...,i received this product early from the seller!...
1,5,*****<br />Numi's Collection Assortment Melang...,*****<br />numi's collection assortment melang...
2,5,"I was very careful not to overcook this pasta,...","i was very careful not to overcook this pasta,..."
3,5,Buying this multi-pack I was misled by the pic...,buying this multi-pack i was misled by the pic...
4,5,These bars are so good! I loved them warmed up...,these bars are so good! i loved them warmed up...


In [12]:
train_df['Text_Processed'] = train_df['Text_Processed'].str.translate(str.maketrans('', '', string.punctuation))
train_df.head()

,Score,Text,Text_Processed
0,5,I received this product early from the seller!...,i received this product early from the seller ...
1,5,*****<br />Numi's Collection Assortment Melang...,br numis collection assortment melange include...
2,5,"I was very careful not to overcook this pasta,...",i was very careful not to overcook this pasta ...
3,5,Buying this multi-pack I was misled by the pic...,buying this multipack i was misled by the pict...
4,5,These bars are so good! I loved them warmed up...,these bars are so good i loved them warmed up ...


In [13]:
import re
def clean_text_with_html_urls(df, text_column='Text_Processed'):
    """
    Cleans text within a pandas DataFrame column, removing HTML anchor tags and URLs,
    and converting the text to lowercase.

    Args:
        df (pd.DataFrame): The DataFrame containing the text data.
        text_column (str): The name of the column containing the text.

    Returns:
        pd.DataFrame: The DataFrame with cleaned text.
    """

    def remove_html_urls(text):
        """Removes HTML anchor tags containing URLs."""
        html_url_pattern = re.compile(r'<a\s+href="http[^"]*">')
        return html_url_pattern.sub(r'', text)

    def remove_url_remainder(text):
        """Removes the remainder of the anchor tag."""
        remainder_pattern = re.compile(r'</a>')
        return remainder_pattern.sub(r'', text)

    def remove_all_urls(text):
        """Combines to remove both the anchor tag and the url"""
        text = remove_html_urls(text)
        text = remove_url_remainder(text)
        return text

    df[text_column] = df[text_column].astype(str).apply(remove_all_urls) #handle nan and other types
    df[text_column] = df[text_column].str.lower()
    return df


In [14]:
train_df = clean_text_with_html_urls(train_df)
train_df.head()

,Score,Text,Text_Processed
0,5,I received this product early from the seller!...,i received this product early from the seller ...
1,5,*****<br />Numi's Collection Assortment Melang...,br numis collection assortment melange include...
2,5,"I was very careful not to overcook this pasta,...",i was very careful not to overcook this pasta ...
3,5,Buying this multi-pack I was misled by the pic...,buying this multipack i was misled by the pict...
4,5,These bars are so good! I loved them warmed up...,these bars are so good i loved them warmed up ...


In [15]:
train_df['Text_Processed'] = train_df['Text_Processed'].str.replace(r'\d+', '', regex=True)
train_df.head()

,Score,Text,Text_Processed
0,5,I received this product early from the seller!...,i received this product early from the seller ...
1,5,*****<br />Numi's Collection Assortment Melang...,br numis collection assortment melange include...
2,5,"I was very careful not to overcook this pasta,...",i was very careful not to overcook this pasta ...
3,5,Buying this multi-pack I was misled by the pic...,buying this multipack i was misled by the pict...
4,5,These bars are so good! I loved them warmed up...,these bars are so good i loved them warmed up ...


In [16]:
import nltk
from nltk.corpus import stopwords
import contractions

nltk.download('stopwords')

def expand_and_remove_stopwords(text):
    expanded_text = contractions.fix(text)
    stop_words = set(stopwords.words('english'))
    words = expanded_text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

train_df['Text_Processed'] = train_df['Text_Processed'].apply(expand_and_remove_stopwords)
train_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Score,Text,Text_Processed
0,5,I received this product early from the seller!...,received product early seller tastey great mid...
1,5,*****<br />Numi's Collection Assortment Melang...,br numis collection assortment melange include...
2,5,"I was very careful not to overcook this pasta,...",careful overcook pasta making sure take bite e...
3,5,Buying this multi-pack I was misled by the pic...,buying multipack misled picture whole hazel nu...
4,5,These bars are so good! I loved them warmed up...,bars good loved warmed definitely think great ...


In [17]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt_tab')
train_df['Text_Processed'] = train_df['Text_Processed'].apply(word_tokenize)
train_df.head()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,Score,Text,Text_Processed
0,5,I received this product early from the seller!...,"[received, product, early, seller, tastey, gre..."
1,5,*****<br />Numi's Collection Assortment Melang...,"[br, numis, collection, assortment, melange, i..."
2,5,"I was very careful not to overcook this pasta,...","[careful, overcook, pasta, making, sure, take,..."
3,5,Buying this multi-pack I was misled by the pic...,"[buying, multipack, misled, picture, whole, ha..."
4,5,These bars are so good! I loved them warmed up...,"[bars, good, loved, warmed, definitely, think,..."


In [18]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk import pos_tag

# Download necessary resources
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

# Function to convert POS tag to WordNet format
def get_wordnet_pos(word):
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)  # Default to NOUN if tag not found

# Apply tokenization, POS tagging, and lemmatization
train_df['Text_Processed'] = train_df['Text_Processed'].apply(lambda x: ' '.join(
    [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in word_tokenize(str(x).lower())]
))

train_df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


,Score,Text,Text_Processed
0,5,I received this product early from the seller!...,"[ 'received ' , 'product ' , 'early ' , 'selle..."
1,5,*****<br />Numi's Collection Assortment Melang...,"[ 'br ' , 'numis ' , 'collection ' , 'assortme..."
2,5,"I was very careful not to overcook this pasta,...","[ 'careful ' , 'overcook ' , 'pasta ' , 'makin..."
3,5,Buying this multi-pack I was misled by the pic...,"[ 'buying ' , 'multipack ' , 'misled ' , 'pict..."
4,5,These bars are so good! I loved them warmed up...,"[ 'bars ' , 'good ' , 'loved ' , 'warmed ' , '..."


In [19]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D, BatchNormalization
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.utils import to_categorical

# Data Preparation
X = train_df['Text_Processed'].values
y = train_df['Score'].values

# One-hot encoding the target labels (Score from 1 to 5)
y = to_categorical(y - 1, num_classes=5)  # Subtracting 1 to align labels from 0 to 4

# Tokenization
max_words = 8000  # Reduced vocabulary size
max_len = 100  # Maximum length of sequences

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(X)

X_seq = tokenizer.texts_to_sequences(X)
X_pad = pad_sequences(X_seq, maxlen=max_len, padding="post", truncating="post")

# Stratified K-Fold Cross Validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

accuracies = []  # List to store accuracy of each fold

# K-Fold Cross Validation
for train_index, val_index in kfold.split(X_pad, np.argmax(y, axis=1)):
    # Split data into training and validation sets
    X_train, X_val = X_pad[train_index], X_pad[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Build Model
    model = Sequential([
        Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
        SpatialDropout1D(0.3),  # Increased SpatialDropout
        LSTM(64, return_sequences=True, kernel_regularizer=l2(0.001)),  # L2 Regularization
        BatchNormalization(),  # Batch Normalization
        LSTM(32, kernel_regularizer=l2(0.001)),  # L2 Regularization
        Dropout(0.6),  # Increased Dropout
        Dense(32, activation='relu', kernel_regularizer=l2(0.001)),  # Regularization in Dense layer
        Dense(5, activation='softmax')  # Output layer for 5 classes (1-5)
    ])

    # Compile Model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Early Stopping Callback
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train Model
    model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val), callbacks=[early_stopping], verbose=1)

    # Evaluate Model
    loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
    accuracies.append(accuracy)

# Print average accuracy across all folds
print(f"Average Accuracy: {np.mean(accuracies) * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/20
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 813s 209ms/step - accuracy: 0.6329 - loss: 1.2249 - val_accuracy: 0.6679 - val_loss: 0.9484
Epoch 2/20
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 889s 216ms/step - accuracy: 0.6790 - loss: 0.9033 - val_accuracy: 0.6850 - val_loss: 0.8675
Epoch 3/20
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 859s 215ms/step - accuracy: 0.6899 - loss: 0.8505 - val_accuracy: 0.7028 - val_loss: 0.8101
Epoch 4/20
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 862s 215ms/step - accuracy: 0.7054 - loss: 0.8043 - val_accuracy: 0.7064 - val_loss: 0.7977
Epoch 5/20
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 834s 216ms/step - accuracy: 0.7136 - loss: 0.7788 - val_accuracy: 0.7095 - val_loss: 0.7941
Epoch 6/20
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 857s 215ms/step - accuracy: 0.7156 - loss: 0.7715 - val_accuracy: 0.7110 - val_loss: 0.7889
Epoch 7/20
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 866s 216ms/step - accuracy: 0.7213 - loss: 0.7587 - val_accuracy: 0.7088 - val_loss: 0.7893
Epoch 8/20
3865/3865 ━━━━━━━━━━━━━━━━━━━━ 868s 217ms/step - ac

KeyboardInterrupt: 

In [20]:
test_df = pd.read_csv("test.csv")
test_df.head()

,Id,Text
0,0,Fog chaser was the best both in flavor and bod...
1,1,"We found this at PF Changs, and it tastes just..."
2,2,Not quite a chocolate bar substitute but delic...
3,3,This is not as tasty as Pamela's Almond Biscot...
4,4,"<a href=""http://www.amazon.com/gp/product/B000..."


In [21]:
test_df['Text_Processed'] = test_df['Text'].str.lower()
test_df.head()

,Id,Text,Text_Processed
0,0,Fog chaser was the best both in flavor and bod...,fog chaser was the best both in flavor and bod...
1,1,"We found this at PF Changs, and it tastes just...","we found this at pf changs, and it tastes just..."
2,2,Not quite a chocolate bar substitute but delic...,not quite a chocolate bar substitute but delic...
3,3,This is not as tasty as Pamela's Almond Biscot...,this is not as tasty as pamela's almond biscot...
4,4,"<a href=""http://www.amazon.com/gp/product/B000...","<a href=""http://www.amazon.com/gp/product/b000..."


In [24]:
import re
def clean_text_with_html_urls(df, text_column='Text_Processed'):
    """
    Cleans text within a pandas DataFrame column, removing HTML anchor tags and URLs,
    and converting the text to lowercase.

    Args:
        df (pd.DataFrame): The DataFrame containing the text data.
        text_column (str): The name of the column containing the text.

    Returns:
        pd.DataFrame: The DataFrame with cleaned text.
    """

    def remove_html_urls(text):
        """Removes HTML anchor tags containing URLs."""
        html_url_pattern = re.compile(r'<a\s+href="http[^"]*">')
        return html_url_pattern.sub(r'', text)

    def remove_url_remainder(text):
        """Removes the remainder of the anchor tag."""
        remainder_pattern = re.compile(r'</a>')
        return remainder_pattern.sub(r'', text)

    def remove_all_urls(text):
        """Combines to remove both the anchor tag and the url"""
        text = remove_html_urls(text)
        text = remove_url_remainder(text)
        return text

    df[text_column] = df[text_column].astype(str).apply(remove_all_urls) #handle nan and other types
    df[text_column] = df[text_column].str.lower()
    return df


In [25]:
test_df = clean_text_with_html_urls(test_df)
test_df.head()

,Id,Text,Text_Processed
0,0,Fog chaser was the best both in flavor and bod...,fog chaser was the best both in flavor and bod...
1,1,"We found this at PF Changs, and it tastes just...","we found this at pf changs, and it tastes just..."
2,2,Not quite a chocolate bar substitute but delic...,not quite a chocolate bar substitute but delic...
3,3,This is not as tasty as Pamela's Almond Biscot...,this is not as tasty as pamela's almond biscot...
4,4,"<a href=""http://www.amazon.com/gp/product/B000...",old mother hubbard special recipe mini puppy b...


In [26]:
test_df['Text_Processed'] = test_df['Text_Processed'].str.translate(str.maketrans('', '', string.punctuation))
test_df.head()

,Id,Text,Text_Processed
0,0,Fog chaser was the best both in flavor and bod...,fog chaser was the best both in flavor and bod...
1,1,"We found this at PF Changs, and it tastes just...",we found this at pf changs and it tastes just ...
2,2,Not quite a chocolate bar substitute but delic...,not quite a chocolate bar substitute but delic...
3,3,This is not as tasty as Pamela's Almond Biscot...,this is not as tasty as pamelas almond biscott...
4,4,"<a href=""http://www.amazon.com/gp/product/B000...",old mother hubbard special recipe mini puppy b...


In [27]:
test_df['Text_Processed'] = test_df['Text_Processed'].str.replace(r'\d+', '', regex=True)
test_df.head()

,Id,Text,Text_Processed
0,0,Fog chaser was the best both in flavor and bod...,fog chaser was the best both in flavor and bod...
1,1,"We found this at PF Changs, and it tastes just...",we found this at pf changs and it tastes just ...
2,2,Not quite a chocolate bar substitute but delic...,not quite a chocolate bar substitute but delic...
3,3,This is not as tasty as Pamela's Almond Biscot...,this is not as tasty as pamelas almond biscott...
4,4,"<a href=""http://www.amazon.com/gp/product/B000...",old mother hubbard special recipe mini puppy b...


In [28]:
test_df['Text_Processed'] = test_df['Text_Processed'].apply(lambda x: ' '.join(word_tokenize(str(x).lower())))

In [29]:
X_test_submission = test_df['Text_Processed'].values
tokenizer.fit_on_texts(X_test_submission)

In [30]:
X_test_seq_submission = tokenizer.texts_to_sequences(X_test_submission)

In [31]:
X_test_pad_submission = pad_sequences(X_test_seq_submission, maxlen=max_len, padding="post", truncating="post")

In [35]:
# Predict class probabilities
predictions = model.predict(X_test_pad_submission)

# Convert probabilities to class labels
y_pred = predictions.argmax(axis=1)  # Get class index with highest probability

# If labels start from 1 (not 0), adjust
y_pred += 1  # Convert 0-based indices to 1-based labels
print(y_pred[:10])

3740/3740 ━━━━━━━━━━━━━━━━━━━━ 72s 19ms/step
[4 5 5 5 5 5 5 5 1 5]


In [33]:
sample_submission = pd.read_csv("sampleSubmission.csv")

# Replace the label column with predictions
sample_submission["Score"] = y_pred  # Ensure the column name matches Kaggle's expected output

# Save as submission.csv
sample_submission.to_csv("okok.csv", index=False)